<a href="https://colab.research.google.com/github/patelami3431/Web-Crawler/blob/master/HLT_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import urllib
import requests
import bs4
from bs4 import BeautifulSoup
import pandas as pd
import re

In [0]:
url = "https://www.vox.com/world/2019/8/20/20813786/wildfire-amazon-rainforest-brazil-siberia"
r = requests.get(url)

In [0]:
b = BeautifulSoup(r.content, 'html.parser')


In [32]:
row = b.find(class_ = 'c-entry-content')
r = row.find_all('p')
for link in r:
  l = link.find('a')
  if l is not None:
    print(l.get('href'))

https://www.bbc.com/news/blogs-trending-49406519
https://twitter.com/hashtag/ActForTheAmazon?src=hash&ref_src=twsrc%5Etfw
https://t.co/P1DrCzQO6x
https://www.euronews.com/2019/08/11/brazil-state-of-amazonas-declares-state-of-emergency-over-rising-number-of-forest-fires
http://apple.co/30n765B
https://www.vox.com/2019/8/1/20750200/siberia-wildfire-russia-fire-smoke-trump-putin
https://www.usatoday.com/story/travel/news/2019/08/19/canary-islands-wildfire-spain-forces-thousands-people-evacuate/2049807001/
https://www.latimes.com/california/story/2019-08-19/california-wildfire-acreage-down
https://www.noaa.gov/news/july-2019-was-hottest-month-on-record-for-planet
https://www.vox.com/2019/7/25/8930325/europe-heat-wave-france-uk-paris-germany-record
https://thebarentsobserver.com/en/arktika/2019/07/worlds-warmest-june-ever-heatwave-raged-across-parts-arctic
https://www.freshplaza.com/article/9111410/exceptional-drought-in-the-canary-islands/
https://public.wmo.int/en/media/news/drought-and-h

In [0]:
u = 'https://www.medicalnewstoday.com/articles/5847.php'
p = requests.get(u)
beautiful = BeautifulSoup(p.content, 'html.parser')


In [0]:
r = beautiful.find_all('p')

In [29]:
#r = beautiful.find(class_ = 'content__article-body from-content-api js-article__body')
url = []
for link in r:
  l = link.find('a')
  if l is not None:
    url.append(l.get('href'))
    
c = 0
url_list = []
for l in url:
  if 'article' and 'http' in l:
    if l not in url_list:
      print(l)
      url_list.append(l)
      c += 1
print(c)


http://www.enable-javascript.com
https://www.medicalnewstoday.com/privacy-policy#vendors
https://www.healthline.com/rights-request
https://www.medicalnewstoday.com/advertising
http://www.medicalnewstoday.com/articles/7379.php
http://www.medicalnewstoday.com/articles/7382.php
http://www.medicalnewstoday.com/articles/180858.php
http://www.medicalnewstoday.com/articles/8749.php
http://www.medicalnewstoday.com/articles/149636.php
http://www.medicalnewstoday.com/articles/149454.php
http://www.medicalnewstoday.com/articles/7380.php
http://www.medicalnewstoday.com/articles/7381.php
http://www.medicalnewstoday.com/articles/149090.php
http://www.sciencedirect.com/science/article/pii/S0002934313000806
https://www.betterhealth.vic.gov.au/health
https://www.weightwatchers.com/about/his/history.aspx
https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5329646/
http://www.atkins.com/Program/Overview/How-and-Why-Atkins-Work.aspx
http://www.rawfoodlife.com/#axzz2jGtlNKVX
http://www.vrg.org/nutshell/vegan.htm
